In [ ]:
!pip install pyeffects

## What is functional programming?

> Functional programming ( FP ) is based on a simple premise with far-reaching implications: we construct our programs using only pure functions—in other words, functions that have no side effects. What are side effects? A function has a side effect if it
does something other than simply return a result, for example:

- Modifying a variable
- Modifying a data structure in place
- Setting a field on an object
- Throwing an exception or halting with an error
- Printing to the console or reading user input
- Reading from or writing to a file
- Drawing on the screen

## What is referential transparency?

> An expression e is referentially transparent if, for all programs p, all occurrences of e in p can be replaced by the result of evaluating e without affecting the meaning of p. A function f is pure if the expression f(x) is referentially transparent for all referentially transparent x.


In [ ]:
from typing import List

def add_numbers(numbers: List[int]) -> int:
    sum = 0
    for n in numbers:
        sum = sum + n
    return sum

add_numbers([1, 5, 6, 8])

In [ ]:
from typing import List

def add_numbers(numbers: List[int]) -> int:
    # if sum = 0, we should be able to replace sum on the RHS with 0 and get the same result
    sum = 0
    for n in numbers:
        sum = 0 + n
    return sum

add_numbers([1, 5, 6, 8])

In [ ]:
# Exercise: Re-write w/o using state

## Dealing with Emptiness: Option

In [ ]:
from pyeffects.Option import *
    
class Name:
    def __init__(self, first_name: str = None, last_name: str = None):
        self.first_name = first_name
        self.last_name = last_name
    
    def get_last_name(self) -> Option:
        return Option.of(self.last_name)
        
    def get_first_name(self) -> Option:
        return Option.of(self.first_name)
        

class Parent:
    def __init__(self, name: Name = None, relationship: str = "mother"):
        self.name = name
        self.relationship = relationship
        
    def get_name(self) -> Option:
        return Option.of(self.name)
        
class Child:
    def __init__(self, name: Name = None, mother: Parent = None, father: Parent = None):
        self.name = name
        self.mother = mother
        self.father = father
        
    def get_name(self) -> Option:
        return Option.of(self.name)
        
    def get_father(self) -> Option:
        return Option.of(self.father)
        
    def get_mother(self) -> Option:
        return Option.of(self.mother)

In [ ]:
def get_mothers_first_name(child: Child) -> str:
    if child and child.mother and child.mother.name:
        return child.mother.name.first_name
    else:
        return None
    
c = Child("child 1", Parent(Name("Mom", "1"), "mother"), Parent(Name("Dad", "1"), "father"))

get_mothers_first_name(c)

In [ ]:
def get_mothers_first_name2(child: Child) -> str:
    return child.get_mother()\
        .flat_map(Parent.get_name)\
        .flat_map(Name.get_first_name)\
        .get_or_else("No name")
    
c = Child("child 1", Parent(Name("Mom", "Mother"), "mother"), Parent(Name("Dad", "Father"), "father"))
c1 = Child("child 2")
# get_mothers_first_name2(c)
get_mothers_first_name2(c1)

## Dealing with Exceptions: Try

In [ ]:
c1_str = """
{
  "name": {
      "first_name": "1st",
      "last_name": "Child"
  },
  "mother": {
      "name": {
          "first_name": "Mother",
          "last_name": "1"
      }
  }
}
"""

# c2_str = """
# {
#   "name": {
#       "first_name": "1st",
#       "last_name": "Child"
#   },
#   "mother": {
#       "name": {
#           "first_name_wrong": "Mother",
#           "last_name": "1"
#       }
#   }
# }
# """

In [ ]:
import json

def load_name(name_dict: dict) -> Name:
    try:
        n = Name(**name_dict)
        return n
    except TypeError as te:
        raise TypeError("Couldn't deserialize Name: " + str(te))

def load_parent_obj(parent_dict: dict, relationship: str) -> Parent:
    n = None
    if 'name' in parent_dict:
        n = load_name(parent_dict['name'])
    return Parent(name = n, relationship = relationship)
        
def load_child_obj(json_str: str) -> Child:
    attributes = json.loads(json_str)
    n, m, f = (None, None, None)
    if 'name' in attributes:
        n = load_name(attributes['name'])
    if 'mother' in attributes:
        m = load_parent_obj(attributes['mother'], 'mother')
    if 'father' in attributes:
        f = load_parent_obj(attributes['father'], 'father')
    return Child(n, m, f)

c1 = load_child_obj(c1_str)
c1.mother.name.first_name

# c2 = load_child_obj(c2_str)

In [ ]:
import json

from pyeffects.Try import *

def load_name(name_dict: dict) -> Try:
    return Try.of(lambda: Name(**name_dict['name']))

def load_parent_obj(parent_dict: dict, relationship: str) -> Parent:
    return load_name(parent_dict)\
        .map(lambda n: Parent(name = n, relationship = relationship))\
        .get_or_else(Parent(relationship = relationship))
        
def load_child_obj(json_str: str) -> Child:
    attributes = json.loads(json_str)
    n = load_name(attributes).get_or_else(None)
    m = Try.of(lambda: load_parent_obj(attributes['mother'], 'mother')).get_or_else(None)
    f = Try.of(lambda: load_parent_obj(attriutes['father'], 'father')).get_or_else(None)
    return Child(n, m, f)

c1 = load_child_obj(c1_str)
c1.mother.name.first_name

c2 = load_child_obj(c2_str)
print(c2.mother.name)